### Create a Feature Group

In [1]:
import pandas as pd
import sagemaker
from time import gmtime, strftime, sleep

import boto3

Configure SageMaker feature store with custom config

In [2]:
from botocore.config import Config

boto_session = boto3.Session()

sm_boto_fs_runtime = boto3.client(service_name='sagemaker-featurestore-runtime', config=Config(
                                     retries = {
                                         'max_attempts': 10,
                                         'mode': 'standard'
                                     }))

sm_boto_client = boto_session.client(
    service_name='sagemaker'
)

sm_session = sagemaker.Session(
    boto_session=boto_session,
    sagemaker_client=sm_boto_client,
    sagemaker_featurestore_runtime_client=sm_boto_fs_runtime)

Create Feature Group Instance

In [3]:
fg_create_time = strftime('%m-%d-%Y-%H-%M-%S', gmtime())
fg_name = 'ingest-fg-'+ fg_create_time

from sagemaker.feature_store.feature_group import FeatureGroup
fg = FeatureGroup(name=fg_name, sagemaker_session=sm_session)

Provision Feature Group

In [4]:
from sagemaker import get_execution_role
df_feature_defs = pd.read_csv('../data/seed.csv')
df_feature_defs['event_date'] = df_feature_defs['event_date'].astype('string')
fg.load_feature_definitions(data_frame=df_feature_defs)
kwargs = dict(
    s3_uri = False,
    record_identifier_name='id',
    event_time_feature_name='event_date',
    role_arn=get_execution_role(),
    enable_online_store=True
)
fg.create(**kwargs)

# wait for feature groups to be created
def wait_for_feature_group_creation_complete(feature_group):
    status = feature_group.describe().get("FeatureGroupStatus")
    while status == "Creating":
        print("Waiting for Feature Group Creation")
        sleep(5)
        status = feature_group.describe().get("FeatureGroupStatus")
    if status != "Created":
        raise RuntimeError(f"Failed to create feature group {feature_group.name}")
    print(f"FeatureGroup {feature_group.name} successfully created.")

wait_for_feature_group_creation_complete(fg)

Waiting for Feature Group Creation
Waiting for Feature Group Creation
Waiting for Feature Group Creation
Waiting for Feature Group Creation
FeatureGroup ingest-fg-06-17-2021-14-46-44 successfully created.


In [5]:
fg_name

'ingest-fg-06-17-2021-14-46-44'